In [2]:
from sklearn.model_selection import StratifiedKFold
import pickle

import os
import pandas as pds

dataname = 'Internet Firewall Data Data Set'
df = pds.read_csv(os.path.join('data',dataname+'.csv'))


In [3]:
X = df.drop(columns = 'Action')
y = df.Action

X.describe()

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
count,65532.000000,65532.000000,65532.000000,65532.000000,6.553200e+04,6.553200e+04,6.553200e+04,6.553200e+04,65532.000000,65532.000000,65532.000000
mean,49391.969343,10577.385812,19282.972761,2671.049930,9.712395e+04,2.238580e+04,7.473815e+04,1.028660e+02,65.833577,41.399530,61.466505
std,15255.712537,18466.027039,21970.689669,9739.162278,5.618439e+06,3.828139e+06,2.463208e+06,5.133002e+03,302.461762,3218.871288,2223.332271
min,0.000000,0.000000,0.000000,0.000000,6.000000e+01,6.000000e+01,0.000000e+00,1.000000e+00,0.000000,1.000000,0.000000
25%,49183.000000,80.000000,0.000000,0.000000,6.600000e+01,6.600000e+01,0.000000e+00,1.000000e+00,0.000000,1.000000,0.000000
50%,53776.500000,445.000000,8820.500000,53.000000,1.680000e+02,9.000000e+01,7.900000e+01,2.000000e+00,15.000000,1.000000,1.000000
75%,58638.000000,15000.000000,38366.250000,443.000000,7.522500e+02,2.100000e+02,4.490000e+02,6.000000e+00,30.000000,3.000000,2.000000
max,65534.000000,65535.000000,65535.000000,65535.000000,1.269359e+09,9.484772e+08,3.208818e+08,1.036116e+06,10824.000000,747520.000000,327208.000000


In [4]:
X

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,177,94,83,2,30,1,1
1,56258,3389,56258,3389,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,25358,6778,18580,31,16,13,18
...,...,...,...,...,...,...,...,...,...,...,...
65527,63691,80,13237,80,314,192,122,6,15,4,2
65528,50964,80,13485,80,4680740,67312,4613428,4675,77,985,3690
65529,54871,445,0,0,70,70,0,1,0,1,0
65530,54870,445,0,0,70,70,0,1,0,1,0


In [4]:
y.value_counts()

allow         37640
deny          14987
drop          12851
reset-both       54
Name: Action, dtype: int64

In [5]:
target_dict = {y : n for n,y in enumerate(y.unique())}
y = y.apply(lambda x : target_dict[x])

ss1 = StratifiedKFold(6, shuffle = True, random_state = 523)
ss2 = StratifiedKFold(5, shuffle = True, random_state = 523)

for train_idx, test_idx in ss1.split(X,y):
    train_X, train_y = X.iloc[train_idx], y.iloc[train_idx]
    test_X, test_y = X.iloc[test_idx], y.iloc[test_idx]
    
    
    for train_idx, valid_idx in ss2.split(train_X,train_y):
        valid_X, valid_y = train_X.iloc[valid_idx], train_y.iloc[valid_idx]
        train_X, train_y = train_X.iloc[train_idx], train_y.iloc[train_idx]
        break
    break

with open(os.path.join('data','Internet.pkl'),'wb') as f:
    pickle.dump(
        dict(
            train_x = train_X.values,
            train_y = train_y.values,
            valid_x = valid_X.values,
            valid_y = valid_y.values,
            test_X = test_X.values,
            test_y = test_y.values,
            columns = train_X.columns,
            unique_y = target_dict
        ),f
    )